# Open Targets Cancer Gene-Disease-MeSH Pipeline v2

This notebook builds the complete crosswalk from raw data sources:

| Step | Source | Output |
|------|--------|--------|
| 1 | Open Targets `disease/*.parquet` | Cancer diseases with MeSH IDs |
| 2 | NLM `d2025.bin` + OT associations | Gene-MeSH crosswalk with levels |
| 3 | NCBI `gene2ensembl.gz` | Final TSV with Entrez IDs |

**Final output columns:**
- `disease_mesh_id` - MeSH descriptor ID
- `gene_entrez_id` - NCBI Entrez Gene ID
- `mesh_level` - Hierarchy depth (2=broad, 9=specific)
- `ot_score` - Open Targets association score
- `evidence_count` - Number of evidence sources


In [14]:
import pandas as pd
import gzip
from pathlib import Path
from collections import defaultdict

DATA_DIR = Path("../data")
OT_DIR = DATA_DIR / "opentargets"

pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 200)

---
## Step 1: Extract Cancer Diseases from Open Targets

Load raw disease parquet files and filter to cancer (EFO_0000616 = neoplasm).

In [15]:
# Load raw disease index
disease_path = OT_DIR / "disease"
disease_files = list(disease_path.glob("**/*.parquet"))
print(f"Loading {len(disease_files)} disease parquet files...")

diseases = pd.concat([pd.read_parquet(f) for f in disease_files])
print(f"Total diseases: {len(diseases):,}")
diseases[['id', 'name', 'ancestors', 'dbXRefs']].head()

Loading 1 disease parquet files...
Total diseases: 46,960


,id,name,ancestors,dbXRefs
0,DOID_0050890,synucleinopathy,"[MONDO_0021179, EFO_0009386, EFO_0000618, EFO_...","[MONDO:0000510, UMLS:C5191670, MEDGEN:1682194,..."
1,DOID_10113,trypanosomiasis,"[EFO_0001067, MONDO_0002428, EFO_0005741]","[MEDGEN:52872, SCTID:78940002, MESH:D014352, I..."
2,DOID_10718,giardiasis,"[MONDO_0043424, EFO_0010282, EFO_0009431, MOND...","[SCTID:10679007, UMLS:C0017536, MESH:D005873, ..."
3,DOID_13406,pulmonary sarcoidosis,"[MONDO_0019751, EFO_0009676, OTAR_0000010, MON...","[ICD10CM:D86.0, MONDO:0001708, NCIt:C34997, NC..."
4,DOID_1947,trichomoniasis,"[MONDO_0002428, EFO_0005741, EFO_0001067]","[ICD10CM:A59, SCTID:56335008, DOID:1947, icd11..."


In [16]:
# Filter to cancer: ancestors must contain EFO_0000616
CANCER_TA = "EFO_0000616"

def has_cancer_ancestor(ancestors):
    if ancestors is None or (isinstance(ancestors, float) and pd.isna(ancestors)):
        return False
    return CANCER_TA in ancestors

cancer_mask = diseases['ancestors'].apply(has_cancer_ancestor)
cancer_diseases = diseases[cancer_mask & (diseases['id'] != CANCER_TA)].copy()

print(f"Cancer diseases: {len(cancer_diseases):,} ({len(cancer_diseases)/len(diseases)*100:.1f}% of total)")

Cancer diseases: 3,395 (7.2% of total)


In [17]:
# Extract MeSH IDs from dbXRefs field
# dbXRefs looks like: ["MeSH:D001943", "OMIM:114480", ...]

def extract_mesh_ids(xrefs):
    if xrefs is None or (isinstance(xrefs, float) and pd.isna(xrefs)):
        return None
    mesh_ids = [ref[5:] for ref in xrefs if ref and ref.lower().startswith('mesh:')]
    return mesh_ids if mesh_ids else None

cancer_diseases['meshIds'] = cancer_diseases['dbXRefs'].apply(extract_mesh_ids)

with_mesh = cancer_diseases['meshIds'].notna().sum()
print(f"With MeSH mapping: {with_mesh:,} ({with_mesh/len(cancer_diseases)*100:.1f}%)")
print(f"Without MeSH: {len(cancer_diseases) - with_mesh:,}")

# Show sample
cancer_diseases[cancer_diseases['meshIds'].notna()][['id', 'name', 'meshIds']].head(10)

With MeSH mapping: 627 (18.5%)
Without MeSH: 2,768


,id,name,meshIds
3073,MONDO_0000430,mature T-cell and NK-cell non-Hodgkin lymphoma,[D016411]
3103,MONDO_0000502,villous adenoma,[D018253]
3231,MONDO_0000920,duodenum cancer,[D004379]
3274,MONDO_0001023,prolymphocytic leukemia,[D015463]
3355,MONDO_0001256,arteriovenous hemangioma/malformation,[D001165]
3388,MONDO_0001340,heart cancer,[D006338]
3408,MONDO_0001398,ureter benign neoplasm,[D014516]
3411,MONDO_0001402,vaginal cancer,[D014625]
3449,MONDO_0001528,vulva cancer,[D014846]
3463,MONDO_0001569,acoustic neuroma,[D009464]


---
## Step 2a: Extract MeSH Hierarchy from d2025.bin

Parse the raw NLM MeSH descriptor file to extract C04.588 (Neoplasms by Site).

In [18]:
# Parse raw MeSH file
mesh_path = DATA_DIR / "mesh" / "d2025.bin"
print(f"Parsing: {mesh_path}")
print(f"Size: {mesh_path.stat().st_size / 1e6:.1f} MB")

def parse_mesh_file(filepath):
    """Parse MeSH ASCII descriptor file into records."""
    records = []
    current = {}
    
    with open(filepath, 'r', encoding='utf-8', errors='replace') as f:
        for line in f:
            line = line.rstrip()
            if line == '*NEWRECORD':
                if current:
                    records.append(current)
                current = {'MN': [], 'UI': None, 'MH': None}
            elif ' = ' in line:
                key, value = line.split(' = ', 1)
                if key == 'MN':  # Tree number (can have multiple)
                    current['MN'].append(value)
                elif key == 'UI':  # Unique ID (e.g., D001943)
                    current['UI'] = value
                elif key == 'MH':  # MeSH Heading name
                    current['MH'] = value
        if current:
            records.append(current)
    
    return records

mesh_records = parse_mesh_file(mesh_path)
print(f"Total MeSH descriptors: {len(mesh_records):,}")

Parsing: ../data/mesh/d2025.bin
Size: 31.4 MB
Total MeSH descriptors: 30,956


In [19]:
# Extract C04.588 hierarchy (Neoplasms by Site)
PREFIX = "C04.588"  # Anatomical site classification

rows = []
for rec in mesh_records:
    if not rec.get('UI') or not rec.get('MH'):
        continue
    for tree_num in rec['MN']:
        if tree_num.startswith(PREFIX) or tree_num == PREFIX:
            level = tree_num.count('.') + 1
            rows.append({
                'mesh_id': rec['UI'],
                'mesh_name': rec['MH'],
                'tree_number': tree_num,
                'level': level
            })

mesh_hierarchy = pd.DataFrame(rows)
print(f"C04.588 hierarchy:")
print(f"  Tree paths: {len(mesh_hierarchy)}")
print(f"  Unique terms: {mesh_hierarchy['mesh_id'].nunique()}")
print(f"  Levels: {mesh_hierarchy['level'].min()}-{mesh_hierarchy['level'].max()}")

mesh_hierarchy.head(15)

C04.588 hierarchy:
  Tree paths: 271
  Unique terms: 236
  Levels: 2-9


,mesh_id,mesh_name,tree_number,level
0,D000008,Abdominal Neoplasms,C04.588.033,3
1,D000306,Adrenal Cortex Neoplasms,C04.588.322.078.265,5
2,D000310,Adrenal Gland Neoplasms,C04.588.322.078,4
3,D000694,Anal Gland Neoplasms,C04.588.083,3
4,D000694,Anal Gland Neoplasms,C04.588.274.476.411.307.790.040.040,9
5,D001005,Anus Neoplasms,C04.588.274.476.411.307.790.040,8
6,D001063,Appendiceal Neoplasms,C04.588.274.476.411.184.290,7
7,D001650,Bile Duct Neoplasms,C04.588.274.120.250,5
8,D001661,Biliary Tract Neoplasms,C04.588.274.120,4
9,D001749,Urinary Bladder Neoplasms,C04.588.945.947.960,5


In [20]:
# Level distribution
print("=== MeSH Level Distribution ===")
print(mesh_hierarchy['level'].value_counts().sort_index())

=== MeSH Level Distribution ===
2     1
3    17
4    67
5    90
6    65
7    20
8    10
9     1
Name: level, dtype: int64


---
## Step 2b: Build Disease-MeSH Crosswalk

Join cancer diseases with MeSH hierarchy.

In [21]:
# Explode meshIds and join with hierarchy
with_mesh_df = cancer_diseases[cancer_diseases['meshIds'].notna()][['id', 'name', 'meshIds']].copy()

# Explode: one row per (disease, meshId)
crosswalk_rows = []
for _, row in with_mesh_df.iterrows():
    for mesh_id in row['meshIds']:
        crosswalk_rows.append({
            'diseaseId': row['id'],
            'diseaseName': row['name'],
            'meshId': mesh_id
        })

disease_mesh_exploded = pd.DataFrame(crosswalk_rows)
print(f"Exploded disease-MeSH pairs: {len(disease_mesh_exploded):,}")

# Join with C04.588 hierarchy (inner join = only keep matches)
disease_mesh_crosswalk = disease_mesh_exploded.merge(
    mesh_hierarchy.rename(columns={'mesh_id': 'meshId'}),
    on='meshId',
    how='inner'
)
print(f"After join with hierarchy: {len(disease_mesh_crosswalk):,}")

# Dedupe: some meshIds have multiple tree paths (different levels)
# Keep most GENERAL (lowest level number) for broader patent coverage
dupes = disease_mesh_crosswalk.duplicated(subset=['diseaseId', 'meshId']).sum()
print(f"Duplicate (diseaseId, meshId) pairs: {dupes}")

disease_mesh_crosswalk = disease_mesh_crosswalk.sort_values('level', ascending=True)
disease_mesh_crosswalk = disease_mesh_crosswalk.drop_duplicates(
    subset=['diseaseId', 'meshId'], 
    keep='first'
)
print(f"After dedupe (keeping most general): {len(disease_mesh_crosswalk):,}")
print(f"  Diseases: {disease_mesh_crosswalk['diseaseId'].nunique()}")
print(f"  MeSH terms: {disease_mesh_crosswalk['meshId'].nunique()}")

disease_mesh_crosswalk.head(10)

Exploded disease-MeSH pairs: 641
After join with hierarchy: 205
Duplicate (diseaseId, meshId) pairs: 24
After dedupe (keeping most general): 181
  Diseases: 181
  MeSH terms: 158


,diseaseId,diseaseName,meshId,mesh_name,tree_number,level
67,EFO_0003769,endocrine neoplasm,D004701,Endocrine Gland Neoplasms,C04.588.322,3
152,EFO_0007491,spleen cancer,D013160,Splenic Neoplasms,C04.588.842,3
165,EFO_1000804,anal gland neoplasm,D000694,Anal Gland Neoplasms,C04.588.083,3
115,EFO_0004198,skin neoplasm,D012878,Skin Neoplasms,C04.588.805,3
111,EFO_0003869,breast neoplasm,D001943,Breast Neoplasms,C04.588.180,3
66,MONDO_0021069,malignant endocrine neoplasm,D004701,Endocrine Gland Neoplasms,C04.588.322,3
109,EFO_0003863,urogenital neoplasm,D014565,Urogenital Neoplasms,C04.588.945,3
17,MONDO_0002334,hematopoietic and lymphoid system neoplasm,D019337,Hematologic Neoplasms,C04.588.448,3
16,EFO_0003824,eye neoplasm,D005134,Eye Neoplasms,C04.588.364,3
15,MONDO_0002236,ocular cancer,D005134,Eye Neoplasms,C04.588.364,3


---
## Step 2c: Load Associations and Build Gene-MeSH Dataset

In [22]:
# Load gene-disease associations
assoc_path = OT_DIR / "association_overall_direct"
assoc_files = list(assoc_path.glob("*.parquet"))
print(f"Loading {len(assoc_files)} association files...")

associations = pd.concat([pd.read_parquet(f) for f in assoc_files])
print(f"Total associations: {len(associations):,}")
associations.head()

Loading 20 association files...
Total associations: 4,492,971


,diseaseId,targetId,score,evidenceCount
0,EFO_0006892,ENSG00000136997,0.001478,1
1,EFO_0006892,ENSG00000196664,0.003696,1
2,EFO_0006893,ENSG00000010610,0.007392,1
3,EFO_0006893,ENSG00000039068,0.005544,2
4,EFO_0006893,ENSG00000080824,0.002217,1


In [23]:
# Filter to diseases with MeSH, join with crosswalk
disease_ids_with_mesh = set(disease_mesh_crosswalk['diseaseId'])
cancer_assoc = associations[associations['diseaseId'].isin(disease_ids_with_mesh)].copy()
print(f"Associations for diseases with MeSH: {len(cancer_assoc):,}")

# Join: add meshId to each association
joined = cancer_assoc.merge(
    disease_mesh_crosswalk[['diseaseId', 'meshId']],
    on='diseaseId',
    how='inner'
)
print(f"After joining with crosswalk: {len(joined):,}")

Associations for diseases with MeSH: 182,018
After joining with crosswalk: 182,018


In [25]:
# Aggregate by (gene, meshId): MAX score, SUM evidenceCount
gene_mesh = joined.groupby(['targetId', 'meshId']).agg({
    'score': 'max',
    'evidenceCount': 'sum'
}).reset_index()

# Add mesh level (min level for meshIds with multiple tree positions)
mesh_levels = mesh_hierarchy.groupby('mesh_id')['level'].min().reset_index()
mesh_levels.columns = ['meshId', 'meshLevel']
gene_mesh = gene_mesh.merge(mesh_levels, on='meshId', how='left')

print(f"Gene-MeSH pairs: {len(gene_mesh):,}")
print(f"  Unique genes: {gene_mesh['targetId'].nunique():,}")
print(f"  Unique MeSH: {gene_mesh['meshId'].nunique()}")

gene_mesh.sort_values('score', ascending=False).head(10)

Gene-MeSH pairs: 174,965
  Unique genes: 20,918
  Unique MeSH: 146


,targetId,meshId,score,evidenceCount,meshLevel
71063,ENSG00000133895,D018761,0.865323,3007,5
114284,ENSG00000165731,D018813,0.859057,1442,5
90548,ENSG00000146648,D002289,0.849510,25065,8
81623,ENSG00000139618,D001943,0.843239,455,3
81820,ENSG00000139687,D012175,0.841107,4819,5
92168,ENSG00000147889,D008545,0.829441,2088,4
3376,ENSG00000012048,D061325,0.827852,8347,4
114285,ENSG00000165731,D018814,0.819577,669,5
17698,ENSG00000083093,D001943,0.813355,84,3
38027,ENSG00000108384,D061325,0.811941,1055,4


---
## Step 3: Map Ensembl to Entrez Gene IDs

Use NCBI gene2ensembl to convert Ensembl IDs to Entrez.

In [26]:
# Load gene2ensembl (human only)
g2e_path = DATA_DIR / "ncbi" / "gene2ensembl.gz"
print(f"Loading: {g2e_path}")

with gzip.open(g2e_path, 'rt') as f:
    g2e = pd.read_csv(f, sep='\t', dtype=str)

# Filter to human (tax_id = 9606)
g2e_human = g2e[g2e['#tax_id'] == '9606'].copy()
g2e_human = g2e_human.rename(columns={
    'GeneID': 'entrezGeneId',
    'Ensembl_gene_identifier': 'ensemblGeneId'
})

# Dedupe to one Entrez per Ensembl
entrez_map = g2e_human[['entrezGeneId', 'ensemblGeneId']].drop_duplicates()
entrez_map = entrez_map.drop_duplicates(subset=['ensemblGeneId'], keep='first')

print(f"Human Ensembl -> Entrez mappings: {len(entrez_map):,}")
entrez_map.head()

Loading: ../data/ncbi/gene2ensembl.gz
Human Ensembl -> Entrez mappings: 38,278


,entrezGeneId,ensemblGeneId
5011659,1,ENSG00000121410
5011660,2,ENSG00000175899
5011662,9,ENSG00000171428
5011668,10,ENSG00000156006
5011669,12,ENSG00000196136


In [27]:
# Map genes
final = gene_mesh.merge(
    entrez_map.rename(columns={'ensemblGeneId': 'targetId'}),
    on='targetId',
    how='left'
)

before = len(final)
final = final.dropna(subset=['entrezGeneId'])
print(f"Mapped to Entrez: {len(final):,}/{before:,} ({len(final)/before*100:.1f}%)")

# Format final 5-column output
final['entrezGeneId'] = final['entrezGeneId'].astype(int)
final = final[['meshId', 'entrezGeneId', 'meshLevel', 'score', 'evidenceCount']]
final.columns = ['disease_mesh_id', 'gene_entrez_id', 'mesh_level', 'ot_score', 'evidence_count']
final = final.sort_values('ot_score', ascending=False).reset_index(drop=True)

print(f"\n=== FINAL OUTPUT ===")
print(f"Rows: {len(final):,}")
print(f"Unique MeSH: {final['disease_mesh_id'].nunique()}")
print(f"Unique genes: {final['gene_entrez_id'].nunique():,}")
print(f"Level range: {final['mesh_level'].min()}-{final['mesh_level'].max()}")
final.head(20)

Mapped to Entrez: 171,856/174,965 (98.2%)

=== FINAL OUTPUT ===
Rows: 171,856
Unique MeSH: 146
Unique genes: 19,275
Level range: 3-8


,disease_mesh_id,gene_entrez_id,mesh_level,ot_score,evidence_count
0,D018761,4221,5,0.865323,3007
1,D018813,5979,5,0.859057,1442
2,D002289,1956,8,0.849510,25065
3,D001943,675,3,0.843239,455
4,D012175,5925,5,0.841107,4819
5,D008545,1029,4,0.829441,2088
6,D061325,672,4,0.827852,8347
7,D018814,5979,5,0.819577,669
8,D001943,79728,3,0.813355,84
9,D061325,5889,4,0.811941,1055


---
## Summary: Data Flow

```
Open Targets disease/*.parquet (46,960 diseases)
    |
    v  [Filter: ancestors contains EFO_0000616]
Cancer diseases (3,395)
    |
    v  [Extract MeSH from dbXRefs]
With MeSH mapping (627, 18.5%)
    |
    +-- NLM d2025.bin --> C04.588 hierarchy (236 terms, levels 2-9)
    |
    v  [Inner join]
Disease-MeSH crosswalk
    |
    +-- OT associations (4.5M)
    |
    v  [Join + aggregate: MAX score, SUM evidence, MIN level]
Gene-MeSH pairs (~175K)
    |
    +-- NCBI gene2ensembl (38K human genes)
    |
    v  [Map Ensembl -> Entrez]
FINAL: gene_disease_mesh_final.tsv (~172K rows, 5 columns)
  - disease_mesh_id, gene_entrez_id, mesh_level, ot_score, evidence_count
```

---
## Analysis: Top Cancer Sites

In [28]:
# Top sites by gene count
mesh_names = mesh_hierarchy[['mesh_id', 'mesh_name']].drop_duplicates()

top_sites = final.groupby('disease_mesh_id').agg({
    'gene_entrez_id': 'nunique',
    'ot_score': 'mean',
    'evidence_count': 'sum'
}).reset_index()
top_sites.columns = ['mesh_id', 'unique_genes', 'mean_score', 'total_evidence']

top_sites = top_sites.merge(mesh_names, on='mesh_id', how='left')
top_sites = top_sites.sort_values('unique_genes', ascending=False)

print("=== TOP 15 CANCER SITES BY GENE COUNT ===")
top_sites[['mesh_id', 'mesh_name', 'unique_genes', 'total_evidence']].head(15)

=== TOP 15 CANCER SITES BY GENE COUNT ===


,mesh_id,mesh_name,unique_genes,total_evidence
38,D006528,"Carcinoma, Hepatocellular",13257,403994
18,D002289,"Carcinoma, Non-Small-Cell Lung",10408,297025
1,D000077192,Adenocarcinoma of Lung,10204,106594
67,D010051,Ovarian Neoplasms,9665,168383
78,D011471,Prostatic Neoplasms,9471,275620
57,D008545,Melanoma,9152,265669
145,D064726,Triple Negative Breast Neoplasms,7396,132748
2,D000077195,Squamous Cell Carcinoma of Head and Neck,6680,72347
4,D000077277,Esophageal Squamous Cell Carcinoma,6544,53899
14,D001943,Breast Neoplasms,6013,58200


---
## EFO Coverage Check

Are we losing important cancers with the EFO_0000616 filter?

In [29]:
# Find cancer-named diseases NOT captured by EFO filter
cancer_pattern = 'cancer|neoplasm|tumor|tumour|carcinoma|sarcoma|lymphoma|leukemia|leukaemia|melanoma|myeloma'
cancer_named = diseases[diseases['name'].str.lower().str.contains(cancer_pattern, na=False, regex=True)].copy()
cancer_named['has_efo'] = cancer_named['ancestors'].apply(has_cancer_ancestor)

captured = cancer_named['has_efo'].sum()
missed = len(cancer_named) - captured

print(f"Diseases with cancer-related names: {len(cancer_named):,}")
print(f"  Captured by EFO filter: {captured:,} ({captured/len(cancer_named)*100:.1f}%)")
print(f"  Not captured: {missed:,}")

# What are the missed ones?
missed_df = cancer_named[~cancer_named['has_efo']]
print(f"\n=== MISSED BY ONTOLOGY PREFIX ===")
print(missed_df['id'].str.split('_').str[0].value_counts())

Diseases with cancer-related names: 2,732
  Captured by EFO filter: 2,333 (85.4%)
  Not captured: 399

=== MISSED BY ONTOLOGY PREFIX ===
OBA         247
MONDO        74
EFO          57
Orphanet     11
HP           10
Name: id, dtype: int64


In [30]:
# Check association impact
missed_ids = set(missed_df['id'])
lost_assocs = associations[associations['diseaseId'].isin(missed_ids)]

print(f"\n=== ASSOCIATION IMPACT ===")
print(f"Total associations: {len(associations):,}")
print(f"Lost by EFO filter: {len(lost_assocs):,} ({len(lost_assocs)/len(associations)*100:.2f}%)")
print("\n=> Most 'lost' are phenotypes (HP_*), measurements, or umbrella terms")
print("=> Actual cancer diseases are captured")


=== ASSOCIATION IMPACT ===
Total associations: 4,492,971
Lost by EFO filter: 39,358 (0.88%)

=> Most 'lost' are phenotypes (HP_*), measurements, or umbrella terms
=> Actual cancer diseases are captured
